This is a test notebook comparing the error of the approximation of the volume of window using:

    1- Basic MC with a Poisson point process
    2- MC with Pushed point process
    3- Delyon and Portier leave one out MC estimator
    4- Delyon and Portier corrected leave one out MC estimator

To do: 
    
    1- Approximate Delyon and Portier bandwidth parameter using their approach
    2- Plot the mean square error instead of the standard deviations since Delyon and Portier MC are not unbiased
    3- Mimic Delyon and Portier strategy used to find the bandwidth parameter to find the parameters of push (math:`\epsilon` and t)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 24
mpl.rcParams['ytick.labelsize'] = 24
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=16)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [ ]:
# to delete this cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from sklearn.linear_model import LinearRegression

def regression_line(intenisty, std):
    log_intensity = np.log([intensity]).T
    log_std = np.log([std]).T
    std_reg = LinearRegression().fit(log_intensity, log_std)
    return log_intensity*std_reg.coef_ + std_reg.intercept_
    

In [ ]:
# slope
print("poisson", LinearRegression().fit(log_intensity, np.log(mc_std_poisson)).coef_)
print("push", LinearRegression().fit(log_intensity, np.log(mc_std_push)).coef_)
print("delpor", LinearRegression().fit(log_intensity, np.log(mc_std_delpor)).coef_)
print("delpor_c", LinearRegression().fit(log_intensity, np.log(mc_std_delpor_c)).coef_)


In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess
from GPPY.numerical_integration import monte_carlo_integration

d = 2
simu_window = BoxWindow([[-2, 2]]*d)
volume_simu_window = simu_window.volume
support_window = BallWindow(center=[0]*d, radius=1)
f = lambda x: np.array([int(support_window.indicator_function(p) == True) for p in x])
exact_integration = support_window.volume
samples_nb = 40
intensity = [ 5, 10, 20, 30, 40, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
w = volume_simu_window
mc_std_poisson, mc_std_push,  error_poisson, error_push = [], [], [], []
mc_std_delpor, mc_std_delpor_c, error_delpor_c, error_delpor = [], [], [], []
for i in intensity:
    print(i, i*volume_simu_window)
    process = HomogeneousPoissonPointProcess(i)
    poisson = [process.generate_sample(window=simu_window) for _ in range(samples_nb)]
    poisson_pattern = [PointPattern(p, simu_window, i) for p in poisson]
    gpp_process = [GravityPointProcess(p) for p in poisson_pattern]
    volume_basin = gpp_process[0].allocation_basin_volume
    if __name__ == "__main__":
        push = [g.pushed_point_process(epsilon=volume_basin/100,
                                    stop_time=9) for g in gpp_process]
    #MC poisson
    mc_poisson = [monte_carlo_integration(f, p, w) for p in poisson]
    error_poisson.append(exact_integration - np.array(mc_poisson))
    mc_std_poisson.append(stat.stdev(mc_poisson))
    #MC push
    mc_push = [monte_carlo_integration(f, p, w) for p in push]
    error_push.append(exact_integration - np.array(mc_push))
    mc_std_push.append(stat.stdev(mc_push))
    #MC delpor
    mc_delpor = [delyon_portier_integration(f, p, bandwidth=2) for p in poisson]
    error_delpor.append(exact_integration - np.array(mc_delpor))
    mc_std_delpor.append(stat.stdev(mc_delpor))
    #MC delpor corrected
    mc_delpor_c = [delyon_portier_integration(f, p, bandwidth=1.5, correction=True) for p in poisson]
    error_delpor_c.append(exact_integration - np.array(mc_delpor_c))
    mc_std_delpor_c.append(stat.stdev(mc_delpor_c))
    


In [ ]:
# slope
log_intensity = np.log([intensity]).T

print("poisson", LinearRegression().fit(log_intensity, np.log(mc_std_poisson)).coef_)
print("push", LinearRegression().fit(log_intensity, np.log(mc_std_push)).coef_)
print("delpor", LinearRegression().fit(log_intensity, np.log(mc_std_delpor)).coef_)
print("delpor_c", LinearRegression().fit(log_intensity, np.log(mc_std_delpor_c)).coef_)


In [ ]:
import matplotlib.pyplot as plt

log_intensity = np.log([intensity]).T

N = np.array(intensity)*volume_simu_window
fig, axis = plt.subplots(1,2, figsize=(15,6))
axis[0].plot(log_intensity, np.log(mc_std_poisson), 'b.', label="MC poisson")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_poisson), 'b')
axis[0].plot(log_intensity, np.log(mc_std_push), 'k.', label="MC push")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_push), 'k')
axis[0].plot(log_intensity, np.log(mc_std_delpor), 'r.', label="MC delpor")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_delpor), 'r')
axis[0].plot(log_intensity, np.log(mc_std_delpor_c), 'g.', label="MC delpor corrected")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_delpor_c), 'g')
axis[0].plot(log_intensity, np.log(1/np.sqrt(intensity)), 'grey', label=r"$\log(1/\sqrt{\rho})$")
axis[0].set_title("d=%s" %d)

axis[0].set_xlabel(r"Intensity $log(\rho)$")
axis[0].set_ylabel("log(std)")
axis[0].legend()
axis[1].plot(intensity, error_poisson, 'b.')
axis[1].plot(intensity[0], error_poisson[0][0], 'b.', label="poisson")
axis[1].plot(np.array(intensity) +1, error_push, 'k.')
axis[1].plot(np.array(intensity[0]) +1, error_push[0][0], 'k.', label="push")
axis[1].plot(np.array(intensity) + 2, error_delpor, 'r.')
axis[1].plot(np.array(intensity[1]) +2, error_delpor[1][0], 'r.', label="DelPor")
axis[1].plot(np.array(intensity) + 3, error_delpor_c, 'g.')
axis[1].plot(np.array(intensity[1]) +3, error_delpor_c[1][0], 'g.', label="DelPor corrected")
axis[1].set_xlabel("Intensity")
axis[1].set_ylabel("Error")
axis[1].legend()
plt.show()

In [ ]:
# d=3
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess
from GPPY.numerical_integration import monte_carlo_integration

d = 3
simu_window = BoxWindow([[-1.8, 1.8]]*d)
volume_simu_window = simu_window.volume
support_window = BallWindow(center=[0]*d, radius=1)
f = lambda x: np.array([int(support_window.indicator_function(p) == True) for p in x])
exact_integration = support_window.volume
samples_nb = 20
intensity = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 80]
w = volume_simu_window
mc_std_poisson, mc_std_push,  error_poisson, error_push = [], [], [], []
mc_std_delpor, mc_std_delpor_c, error_delpor_c, error_delpor = [], [], [], []
for i in intensity:
    print(i, i*volume_simu_window)
    process = HomogeneousPoissonPointProcess(i)
    poisson = [process.generate_sample(window=simu_window) for _ in range(samples_nb)]
    poisson_pattern = [PointPattern(p, simu_window, i) for p in poisson]
    gpp_process = [GravityPointProcess(p) for p in poisson_pattern]
    volume_basin = gpp_process[0].allocation_basin_volume
    if __name__ == "__main__":
        push = [g.pushed_point_process(epsilon=volume_basin/100,
                                    stop_time=9) for g in gpp_process]
    #MC poisson
    mc_poisson = [monte_carlo_integration(f, p, w) for p in poisson]
    error_poisson.append(exact_integration - np.array(mc_poisson))
    mc_std_poisson.append(stat.stdev(mc_poisson))
    #MC push
    mc_push = [monte_carlo_integration(f, p, w) for p in push]
    error_push.append(exact_integration - np.array(mc_push))
    mc_std_push.append(stat.stdev(mc_push))
    #MC delpor
    mc_delpor = [delyon_portier_integration(f, p, bandwidth=2) for p in poisson]
    error_delpor.append(exact_integration - np.array(mc_delpor))
    mc_std_delpor.append(stat.stdev(mc_delpor))
    #MC delpor corrected
    mc_delpor_c = [delyon_portier_integration(f, p, bandwidth=1.5, correction=True) for p in poisson]
    error_delpor_c.append(exact_integration - np.array(mc_delpor_c))
    mc_std_delpor_c.append(stat.stdev(mc_delpor_c))
    


In [ ]:
# slope
log_intensity = np.log([intensity]).T
print("poisson", LinearRegression().fit(log_intensity, np.log(mc_std_poisson)).coef_)
print("push", LinearRegression().fit(log_intensity, np.log(mc_std_push)).coef_)
print("delpor", LinearRegression().fit(log_intensity, np.log(mc_std_delpor)).coef_)
print("delpor_c", LinearRegression().fit(log_intensity, np.log(mc_std_delpor_c)).coef_)


In [ ]:
import matplotlib.pyplot as plt


N = np.array(intensity)*volume_simu_window
fig, axis = plt.subplots(1,2, figsize=(15,6))
axis[0].plot(log_intensity, np.log(mc_std_poisson), 'b.', label="MC poisson")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_poisson), 'b')
axis[0].plot(log_intensity, np.log(mc_std_push), 'k.', label="MC push")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_push), 'k')
axis[0].plot(log_intensity, np.log(mc_std_delpor), 'r.', label="MC delpor")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_delpor), 'r')
axis[0].plot(log_intensity, np.log(mc_std_delpor_c), 'g.', label="MC delpor corrected")
axis[0].plot(log_intensity, regression_line(intensity, mc_std_delpor_c), 'g')
axis[0].plot(log_intensity, np.log(1/np.sqrt(intensity)), 'grey', label=r"$\log(1/\sqrt{\rho})$")
axis[0].set_title("d=3")

axis[0].set_xlabel(r"Intensity $log(\rho)$")
axis[0].set_ylabel("log(std)")
axis[0].legend()
axis[1].plot(intensity, error_poisson, 'b.')
axis[1].plot(intensity[0], error_poisson[0][0], 'b.', label="poisson")
axis[1].plot(np.array(intensity) +1, error_push, 'k.')
axis[1].plot(np.array(intensity[0]) +1, error_push[0][0], 'k.', label="push")
axis[1].plot(np.array(intensity) + 2, error_delpor, 'r.')
axis[1].plot(np.array(intensity[1]) +2, error_delpor[1][0], 'r.', label="DelPor")
axis[1].plot(np.array(intensity) + 3, error_delpor_c, 'g.')
axis[1].plot(np.array(intensity[1]) +3, error_delpor_c[1][0], 'g.', label="DelPor corrected")
axis[1].set_xlabel("Intensity")
axis[1].set_ylabel("Error")
axis[1].legend()
plt.show()

# Stopping time selection for pushed 

In [ ]:
# pushed function of time
# d=3
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess
from GPPY.numerical_integration import monte_carlo_integration

d = 2
simu_window = BoxWindow([[-1.8, 1.8]]*d)
volume_simu_window = simu_window.volume
support_window = BallWindow(center=[0]*d, radius=1)
f = lambda x: np.array([int(support_window.indicator_function(p) == True) for p in x])
exact_integration = support_window.volume
samples_nb = 40
intensity = 50
w = volume_simu_window
mc_std_poisson, mc_std_push,  error_poisson, error_push = [], [], [], []
push_list =[]

t_max=20
process = HomogeneousPoissonPointProcess(intensity)
poisson = [process.generate_sample(window=simu_window) for _ in range(samples_nb)]
poisson_pattern = [PointPattern(p, simu_window, i) for p in poisson]
gpp_process = [GravityPointProcess(p) for p in poisson_pattern]
volume_basin = gpp_process[0].allocation_basin_volume
for t in range(1, t_max):
    print(t)
    if __name__ == "__main__":
        push = [g.pushed_point_process(epsilon=volume_basin/100,
                                    stop_time=t) for g in gpp_process]
    push_list.append(push[0])
    #MC poisson
    mc_poisson = [monte_carlo_integration(f, p, w) for p in poisson]
    error_poisson.append(exact_integration - np.array(mc_poisson))
    mc_std_poisson.append(stat.stdev(mc_poisson))
    #MC push
    mc_push = [monte_carlo_integration(f, p, w) for p in push]
    error_push.append(exact_integration - np.array(mc_push))
    mc_std_push.append(stat.stdev(mc_push))



In [ ]:
import matplotlib.pyplot as plt

t_stop = list(range(1,t_max))
N = np.array(intensity)*volume_simu_window
fig, axis = plt.subplots(1,2, figsize=(15,6))
axis[0].plot(t_stop, mc_std_poisson, 'b.', label="MC poisson")
axis[0].plot(t_stop, mc_std_push, 'k.', label="MC push")
axis[0].set_title("d=2")
axis[0].set_xlabel("t")
axis[0].set_ylabel("std")
axis[0].legend()
axis[1].plot(t_stop, error_poisson, 'b.')
axis[1].plot(t_stop[0], error_poisson[0][0], 'b.', label="poisson")
axis[1].plot(np.array(t_stop) +0.2, error_push, 'k.')
axis[1].plot(np.array(t_stop[0]) +0.2, error_push[0][0], 'k.', label="push")
axis[1].set_xlabel("t")
axis[1].set_ylabel("Error")
axis[1].legend()
plt.show()

In [ ]:

# d=3
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess
from GPPY.numerical_integration import monte_carlo_integration

d = 3
simu_window = BoxWindow([[-1.8, 1.8]]*d)
volume_simu_window = simu_window.volume
support_window = BallWindow(center=[0]*d, radius=1)
f = lambda x: np.array([int(support_window.indicator_function(p) == True) for p in x])
exact_integration = support_window.volume
samples_nb = 40
intensity = 50
w = volume_simu_window
mc_std_poisson, mc_std_push,  error_poisson, error_push = [], [], [], []
push_list =[]

t_max=20
process = HomogeneousPoissonPointProcess(intensity)
poisson = [process.generate_sample(window=simu_window) for _ in range(samples_nb)]
poisson_pattern = [PointPattern(p, simu_window, i) for p in poisson]
gpp_process = [GravityPointProcess(p) for p in poisson_pattern]
volume_basin = gpp_process[0].allocation_basin_volume
for t in range(1, t_max):
    print(t)
    if __name__ == "__main__":
        push = [g.pushed_point_process(epsilon=volume_basin/100,
                                    stop_time=t) for g in gpp_process]
    push_list.append(push[0])
    #MC poisson
    mc_poisson = [monte_carlo_integration(f, p, w) for p in poisson]
    error_poisson.append(exact_integration - np.array(mc_poisson))
    mc_std_poisson.append(stat.stdev(mc_poisson))
    #MC push
    mc_push = [monte_carlo_integration(f, p, w) for p in push]
    error_push.append(exact_integration - np.array(mc_push))
    mc_std_push.append(stat.stdev(mc_push))



In [ ]:
import matplotlib.pyplot as plt

t_stop = list(range(1,t_max))
N = np.array(intensity)*volume_simu_window
fig, axis = plt.subplots(1,2, figsize=(15,6))
axis[0].plot(t_stop[:13], mc_std_poisson, 'b.', label="MC poisson")
axis[0].plot(t_stop[:13], mc_std_push, 'k.', label="MC push")
axis[0].set_title("d=%i"%d)
axis[0].set_xlabel("t")
axis[0].set_ylabel("std")
axis[0].legend()
axis[1].plot(t_stop[:13], error_poisson, 'b.')
axis[1].plot(t_stop[0], error_poisson[0][0], 'b.', label="poisson")
axis[1].plot(np.array(t_stop[:13]) +0.2, error_push, 'k.')
axis[1].plot(np.array(t_stop[0]) +0.2, error_push[0][0], 'k.', label="push")
axis[1].set_xlabel("t")
axis[1].set_ylabel("Error")
axis[1].legend()
plt.show()

In [ ]:
fig, axis = plt.subplots(figsize=(6,6))
plt.plot(poisson[0][:, 0], poisson[0][:,1])
plt.show
for i in range(1,t_max):
    plt.plot(push[t][:,0], push[t][:,1], 'b.')
    plt.show()

In [ ]:
push

# Bandwidth selection for Delyon and Portier 

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess
from GPPY.numerical_integration import monte_carlo_integration

d = 2
simu_window = BoxWindow([[-2, 2]]*d)
volume_simu_window = simu_window.volume
support_window = BallWindow(center=[0]*d, radius=1)
f = lambda x: np.array([int(support_window.indicator_function(p) == True) for p in x])
exact_integration = support_window.volume
samples_nb = 40
intensity = 60
w = volume_simu_window
mc_std_poisson, error_poisson =  [], []
mc_std_delpor, mc_std_delpor_c, error_delpor_c, error_delpor = [], [], [], []
process = HomogeneousPoissonPointProcess(intensity)
poisson = [process.generate_sample(window=simu_window) for _ in range(samples_nb)]
poisson_pattern = [PointPattern(p, simu_window, intensity) for p in poisson]

bandwidth = [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4, 5]
for h in bandwidth:
    #MC poisson
    mc_poisson = [monte_carlo_integration(f, p, w) for p in poisson]
    error_poisson.append(exact_integration - np.array(mc_poisson))
    mc_std_poisson.append(stat.stdev(mc_poisson))
    #MC delpor
    mc_delpor = [delyon_portier_integration(f, p, bandwidth=h) for p in poisson]
    error_delpor.append(exact_integration - np.array(mc_delpor))
    mc_std_delpor.append(stat.stdev(mc_delpor))
    #MC delpor corrected
    mc_delpor_c = [delyon_portier_integration(f, p, bandwidth=h, correction=True) for p in poisson]
    error_delpor_c.append(exact_integration - np.array(mc_delpor_c))
    mc_std_delpor_c.append(stat.stdev(mc_delpor_c))
    


In [ ]:
import matplotlib.pyplot as plt


N = np.array(intensity)*volume_simu_window
fig, axis = plt.subplots(1,2, figsize=(15,6))
axis[0].plot(bandwidth, mc_std_poisson, 'b.', label="MC poisson")
axis[0].plot(bandwidth, mc_std_delpor, 'r.', label="MC delpor")
axis[0].plot(bandwidth, mc_std_delpor_c, 'g.', label="MC delpor corrected")
axis[0].set_title("d=%i" %d)

axis[0].set_xlabel(r"bandwidth")
axis[0].set_ylabel("std")
axis[0].legend()
axis[1].plot(bandwidth, error_poisson, 'b.')
axis[1].plot(bandwidth, error_poisson[0][0], 'b.', label="poisson")
axis[1].plot(np.array(bandwidth) +0.01, error_delpor, 'r.')
axis[1].plot(np.array(bandwidth[1]) +0.01, error_delpor[1][0], 'r.', label="DelPor")
axis[1].plot(np.array(bandwidth) + 0.02, error_delpor_c, 'g.')
axis[1].plot(np.array(bandwidth[1]) +0.02, error_delpor_c[1][0], 'g.', label="DelPor corrected")
axis[1].set_xlabel("bandwidth")
axis[1].set_ylabel("Error")
axis[1].legend()
plt.show()

In [ ]:
kernel((x - points_[1])/h)

In [ ]:
33/(32*np.pi)

In [ ]:
leave_one_out_kernel_estimator(1, x, points, 2)

In [ ]:
from GPPY.numerical_integration import correction_delyon_portier
correction_delyon_portier(1, x, points, 2)